# Exercise 5 - Image analysis and compression

In [3]:
# Group Number: 03
# Group Members: Kurt Kilian Eifler, Moritz Ladwig, Simon Grünewald

#loading a few python modules used for image processing, io and plotting

import numpy as np
import numpy.matlib
from matplotlib import pyplot as plt
from scipy import ndimage
from scipy import misc
from scipy import fftpack
import skimage
import imageio

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# from accum import accum # scheinbar nicht verfügbar in Python (stammt aus Mathlab)

# for 16 bit PNG support in imageio if needed, imageio.plugins.freeimage.download() might need to be run once per system (or exr/hdr/pgm/ppm support)

In [5]:
# accum Workaround
# gefunden auf: https://scipy-cookbook.readthedocs.io/items/AccumarrayLike.html (Abruf: 6.7.2020)
from itertools import product

def accum(accmap, a, func=None, size=None, fill_value=0, dtype=None):
    """
    An accumulation function similar to Matlab's `accumarray` function.

    Parameters
    ----------
    accmap : ndarray
        This is the "accumulation map".  It maps input (i.e. indices into
        `a`) to their destination in the output array.  The first `a.ndim`
        dimensions of `accmap` must be the same as `a.shape`.  That is,
        `accmap.shape[:a.ndim]` must equal `a.shape`.  For example, if `a`
        has shape (15,4), then `accmap.shape[:2]` must equal (15,4).  In this
        case `accmap[i,j]` gives the index into the output array where
        element (i,j) of `a` is to be accumulated.  If the output is, say,
        a 2D, then `accmap` must have shape (15,4,2).  The value in the
        last dimension give indices into the output array. If the output is
        1D, then the shape of `accmap` can be either (15,4) or (15,4,1) 
    a : ndarray
        The input data to be accumulated.
    func : callable or None
        The accumulation function.  The function will be passed a list
        of values from `a` to be accumulated.
        If None, numpy.sum is assumed.
    size : ndarray or None
        The size of the output array.  If None, the size will be determined
        from `accmap`.
    fill_value : scalar
        The default value for elements of the output array. 
    dtype : numpy data type, or None
        The data type of the output array.  If None, the data type of
        `a` is used.

    Returns
    -------
    out : ndarray
        The accumulated results.

        The shape of `out` is `size` if `size` is given.  Otherwise the
        shape is determined by the (lexicographically) largest indices of
        the output found in `accmap`.


    Examples
    --------
    >>> from numpy import array, prod
    >>> a = array([[1,2,3],[4,-1,6],[-1,8,9]])
    >>> a
    array([[ 1,  2,  3],
           [ 4, -1,  6],
           [-1,  8,  9]])
    >>> # Sum the diagonals.
    >>> accmap = array([[0,1,2],[2,0,1],[1,2,0]])
    >>> s = accum(accmap, a)
    array([9, 7, 15])
    >>> # A 2D output, from sub-arrays with shapes and positions like this:
    >>> # [ (2,2) (2,1)]
    >>> # [ (1,2) (1,1)]
    >>> accmap = array([
            [[0,0],[0,0],[0,1]],
            [[0,0],[0,0],[0,1]],
            [[1,0],[1,0],[1,1]],
        ])
    >>> # Accumulate using a product.
    >>> accum(accmap, a, func=prod, dtype=float)
    array([[ -8.,  18.],
           [ -8.,   9.]])
    >>> # Same accmap, but create an array of lists of values.
    >>> accum(accmap, a, func=lambda x: x, dtype='O')
    array([[[1, 2, 4, -1], [3, 6]],
           [[-1, 8], [9]]], dtype=object)
    """

    # Check for bad arguments and handle the defaults.
    if accmap.shape[:a.ndim] != a.shape:
        raise ValueError("The initial dimensions of accmap must be the same as a.shape")
    if func is None:
        func = np.sum
    if dtype is None:
        dtype = a.dtype
    if accmap.shape == a.shape:
        accmap = np.expand_dims(accmap, -1)
    adims = tuple(range(a.ndim))
    if size is None:
        size = 1 + np.squeeze(np.apply_over_axes(np.max, accmap, axes=adims))
    size = np.atleast_1d(size)

    # Create an array of python lists of values.
    vals = np.empty(size, dtype='O')
    for s in product(*[range(k) for k in size]):
        vals[s] = []
    for s in product(*[range(k) for k in a.shape]):
        indx = tuple(accmap[s])
        val = a[s]
        vals[indx].append(val)

    # Create the output array.
    out = np.empty(size, dtype=dtype)
    for s in product(*[range(k) for k in size]):
        if vals[s] == []:
            out[s] = fill_value
        else:
            out[s] = func(vals[s])

    return out

In [ ]:
# Load our favorite test image
peppersImg = imageio.imread('peppers.png')
peppersImg = np.float64( peppersImg ) / 255

# Create standard 75% color bars with the width of the peppers image and the following height:
bars_and_gradient_height = 64

# Your code here
# Your code here
colorBar75 = # Your code here

# Create a gradient from 0...1 with the width of the peppers image and the same height as the color bars
gradient = # Your code here

# Display shapes to see if they fit together
print( peppersImg.shape )
print( colorBar75.shape )
print( gradient.shape )

In [ ]:
# Concatenate and show Image
plt.subplot(1,2,1)
img = np.concatenate( ( peppersImg, colorBar75, gradient), axis=0 )
plt.imshow( img );
plt.title("Your Image")
plt.subplot(1,2,2)
plt.imshow( plt.imread( "exercise_05_results_for_reference/peppers_bar_gradient.png" ), interpolation = 'catrom' );
plt.title("Reference Image\nwith color bars\nand gradient");

## 5.1 Histogram

Histograms are useful to see the distribution of code values between blacks, mid tones and highlights and to eliminate color casts. They can also be used to detect reduced quantization.

In [ ]:
# Finish the function `calculateHistogramm` by looping over all bins and not using the Python histogram functions.
def calculateHistogramm(img, bins = 256, minval = 0.0, maxval = 1.0):
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    return result

In [ ]:
# Apply the ASC-CDL grading controls from 4.2 to get an intuition how the histogram behaves.
# Your code here
# Your code here
# Your code here
# Your code here
gradedImg = # Your code here

pepper_hist = calculateHistogramm( gradedImg )
hist_range  = np.arange(0, pepper_hist.shape[0])

# Display
plt.figure(figsize=(20,5))
plt.subplot(2,2,(1,3))
plt.imshow( gradedImg )
plt.subplot(2,2,2)
img = np.concatenate( ( peppersImg, colorBar75, gradient), axis=0 )
plt.bar(hist_range, pepper_hist);
plt.title("Your Histogram")
plt.subplot(2,2,4)
plt.imshow( plt.imread( "exercise_05_results_for_reference/histogram.png" ), interpolation = 'catrom' );
plt.axis('off')
plt.title("Reference Histogram if no grading is applied");

In [ ]:

# Play with the histogram resolution to detect that peppers.png seems to be 8 bits.
# Increasing the number of bins is one way to do this, but setting minval and maxval to appropriate numbers may be more clever. 
# Your code here
# Your code here
# Your code here
# Your code here
# Your code here
# Your code here
# Your code here
# Your code here


In [ ]:
# Try to hide the fact that peppers.png is only 8bits by resizing it.
# What may be a better choice than resizing? Would this solve visible quantization artifacts in the image?
# Your code here
# Your code here
# Your code here
# Your code here
# Your code here
# Your code here
# Your code here
# Your code here

## 5.2 Waveform

A waveform monitor can be thought as being histogram per column, but the height of the histogram bar is now shown as pixel intensity. 

In [ ]:
# Finish the function `calculateWaveform`.
def calculateWaveform(img, bins = 256, minval = 0.0, maxval = 1.0):
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    return np.flip(result, 0)

In [ ]:
# Apply the grading controls from 4.2 and get an intuition how the waveform behaves.
# Your code here
# Your code here
# Your code here
# Your code here
gradedImg = # Your code here

wf = calculateWaveform(gradedImg, bins=256)
plt.subplot(1,2,1)
plt.imshow( np.concatenate( ( gradedImg, wf), axis=0 ) );
plt.title("Your image");
plt.subplot(1,2,2)
plt.imshow( plt.imread( "exercise_05_results_for_reference/img_and_waveform.png" ), interpolation = 'catrom' );
plt.axis('off')
plt.title("Reference Image and Waveform\nif no grading is applied");

In [ ]:
# Change the bins to 512. How can you detect limited tonal resolution using a waveform? Did you already observe this in grading?
# Your code here
# Your code here
# Your code here
# Your code here

## 5.3 Vectorscope
Colorist often use the vectorscope to adjust color balance. 

In [ ]:
# Finish the function `calculateVectorscope`.
# Hint: For calculating the 2D histogram ‘accum’ will be much faster compared to two loops
def calculateVectorscope(img, bins=256):
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    # Your code here
    return np.flip(result, axis=0)

In [ ]:
# Add some noise to the image before feeding it into the vectorscope so that the color bars do not only end up at the same pixel position.
noise_img = img + (np.random.randn(img.shape[0], img.shape[1], 3) - 0.5) / 256

vectorscopeOutput = calculateVectorscope( noise_img )

plt.subplot(1,2,1)
plt.imshow(np.log2(vectorscopeOutput + 1) / np.log2(np.amax( vectorscopeOutput ) + 1), cmap='gray', vmin=0, vmax=1.0)
plt.title("Your Vectorscope");
plt.subplot(1,2,2)
plt.imshow( plt.imread( "exercise_05_results_for_reference/vectorscope.png" ), interpolation = 'catrom' );
plt.axis('off')
plt.title("Reference Vectorscope\nif no grading is applied");

In [ ]:
# Apply the grading controls from 4.2 and get an intuition how the vectorscope behaves. Especially try saturation
# Your code here
# Your code here
# Your code here
# Your code here
gradedImg = # Your code here
# Your code here
# Your code here
# Your code here
# Your code here